In [1]:
import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np
from lmfit import  Model, Parameters

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [3]:
equation='fct_velocity'
time_sup = 280
opt_base = {'stop_search_misac': None, 'time_sup': 280,
            'param_fit': None, 'inde_vars': None,
            'step_fit':2, 'do_whitening': False,
            'before_sacc': 5, 'after_sacc': 15}    

In [ ]:
for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'

    print(observer,timeStr)

    file = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        param_exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    A = ANEMO(param_exp)
    Fit = ANEMO.Fit(param_exp)

    N_trials = param_exp['N_trials']
    N_blocks = param_exp['N_blocks']

    goodness_of_fit = {}

    for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
        goodness_of_fit[s] = []

    for block in range(N_blocks) :

        for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
            goodness_of_fit[s].append([])

        for trial in range(N_trials) :

            print(block, trial)

            trial_data = trial + N_trials*block
            arg = A.arg(data[trial_data], trial=trial, block=block)
            opt = opt_base.copy()
            opt.update(arg)

            velocity_NAN = A.velocity_NAN(**opt, filt='velocity')

            #-------------------------------------------------
            # FIT
            #-------------------------------------------------
            if equation=='fct_velocity' :
                data_trial = velocity_NAN
            else :
                data_trial = A.data_deg(data=arg.data_x, **opt)

            old_latence, old_max, old_anti = A.classical_method.Full(velocity_NAN, arg.TargetOn-arg.t_0)
            f = Fit.Fit_trial(data_trial, equation=equation, value_latence=old_latence, value_max=old_max, value_anti=old_anti, **opt)
            #-------------------------------------------------
            goodness_of_fit['nfev'][block].append(f.nfev)
            goodness_of_fit['residual'][block].append(f.residual)
            goodness_of_fit['chisqr'][block].append(f.chisqr)
            goodness_of_fit['redchi'][block].append(f.redchi)
            goodness_of_fit['aic'][block].append(f.aic)
            goodness_of_fit['bic'][block].append(f.bic)
            
            
    file = os.path.join('parametre', 'goodness_of_fit_%s_fct_velocity_filt_30.pkl'%(observer))
    with open(file, 'wb') as fichier:
        f = pickle.Pickler(fichier)
        f.dump(goodness_of_fit)